In [1]:
import numpy as np
import re
import json
import pandas as pd

In [2]:
model = "gpt-3.5-turbo"

labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

merged_df = labels.merge(weights, on='pairid')

with open(model + "_response_withsys_doubleprompt.json", "r") as file:
    loaded_data = json.load(file)
    
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight
0,1,Altitude,Temperature,DWD,->,0.166
1,2,Altitude,Precipitation,DWD,->,0.166
2,3,Longitude,Temperature,DWD,->,0.167
3,4,Altitude,Sunshine hours,DWD,->,0.166
4,5,Age,Length,Abalone,->,0.143


In [6]:
def add_prediction(merged_df, loaded_data, model):
    merged_df = merged_df.set_index('pairid')
    for response in loaded_data:
        pairid, resid, answer = extract_answers(response)
        print(pairid,resid, answer)
        col = "AtoB" if resid == "1" else "BtoA"
        merged_df.loc[int(pairid),col] = "Yes" if answer == "Yes." else "No" if answer == "No." else answer
        print("Converted:", merged_df.loc[int(pairid),col])
    return merged_df


def extract_answers(s):
    # Compile regex patterns
    
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),_res(\d)")
    
    # Search for pairid
    pairid_match = pairid_pattern.search(str(s))
    pairid = pairid_match.group(1) if pairid_match else None
    resid = pairid_match.group(2) if pairid_match else None
    
    # Search for answer
    ans = s["pair0"+ str(pairid) + ',_res' + str(resid)]["choices"][0]["message"]['content']
    
    return pairid, resid, ans

In [7]:
loaded_data[1]

{'pair0001,_res2': {'id': 'chatcmpl-7zBbx8JdhwOPPiZrHjzaASwHbRzJy',
  'object': 'chat.completion',
  'created': 1694816997,
  'model': 'gpt-3.5-turbo-0613',
  'choices': [{'index': 0,
    'message': {'role': 'assistant', 'content': 'No.'},
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 40, 'completion_tokens': 2, 'total_tokens': 42}}}

In [8]:
results = add_prediction(merged_df, loaded_data, model)

001 1 Yes.
Converted: Yes
001 2 No.
Converted: No
002 1 Yes.
Converted: Yes
002 2 No.
Converted: No
003 1 No.
Converted: No
003 2 No.
Converted: No
004 1 Yes.
Converted: Yes
004 2 No.
Converted: No
005 1 Yes.
Converted: Yes
005 2 No.
Converted: No
006 1 Yes.
Converted: Yes
006 2 No.
Converted: No
007 1 Yes.
Converted: Yes
007 2 No.
Converted: No
008 1 No.
Converted: No
008 2 No.
Converted: No
009 1 Yes.
Converted: Yes
009 2 No.
Converted: No
010 1 Yes.
Converted: Yes
010 2 No.
Converted: No
011 1 Yes.
Converted: Yes
011 2 No.
Converted: No
012 1 Yes.
Converted: Yes
012 2 No.
Converted: No
013 1 Yes.
Converted: Yes
013 2 No.
Converted: No
014 1 Yes.
Converted: Yes
014 2 No.
Converted: No
015 1 Yes.
Converted: Yes
015 2 No.
Converted: No
016 1 Yes.
Converted: Yes
016 2 No.
Converted: No
017 1 No.
Converted: No
017 2 No.
Converted: No
018 1 Yes.
Converted: Yes
018 2 No.
Converted: No
019 1 Yes.
Converted: Yes
019 2 Yes.
Converted: Yes
020 1 Yes.
Converted: Yes
020 2 No.
Converted: No
021 

In [9]:
results["AtoB"].unique()

array(['Yes', 'No'], dtype=object)

In [10]:
results["gt_AtoB"] = np.where(results["groundtruth"] == "->", "Yes", "No")
results["gt_BtoA"] = np.where(results["groundtruth"] == "->", "No", "Yes")
results["correct_AtoB"] = np.where(results["gt_AtoB"] == results["AtoB"], 1, 0)
results["correct_BtoA"] = np.where(results["gt_BtoA"] == results["BtoA"], 1, 0)
wtd_acc = (sum(results["correct_AtoB"]*results["weight"]) + sum(results["correct_BtoA"]*results["weight"])) / (results["weight"].sum()*2)
acc = (sum(results["correct_AtoB"]) + sum(results["correct_BtoA"])) / 108 / 2
print("acc:", acc, "weighted acc:", wtd_acc)

acc: 0.8518518518518519 weighted acc: 0.844526243720816


In [11]:
results

,varA,varB,dataset,groundtruth,weight,AtoB,BtoA,gt_AtoB,gt_BtoA,correct_AtoB,correct_BtoA
pairid,,,,,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,Yes,No,Yes,No,1,1
2,Altitude,Precipitation,DWD,->,0.166,Yes,No,Yes,No,1,1
3,Longitude,Temperature,DWD,->,0.167,No,No,Yes,No,0,1
4,Altitude,Sunshine hours,DWD,->,0.166,Yes,No,Yes,No,1,1
5,Age,Length,Abalone,->,0.143,Yes,No,Yes,No,1,1
...,...,...,...,...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,No,No,Yes,No,0,1
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,Yes,No,Yes,No,1,1
106,time required for one round,voltage,D. Janzing,<-,1.000,No,Yes,No,Yes,1,1
